In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt, gc, os
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv
/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet
/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet


In [29]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import os
import lightgbm
import matplotlib.pyplot as plt
import seaborn as sbn
import numpy as np
import pandas as pd
import warnings
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, RocCurveDisplay, accuracy_score
import shap
import itertools

In [30]:
train = pd.read_parquet("/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet").groupby('customer_ID').tail(4)
test = pd.read_parquet("/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet").groupby('customer_ID').tail(4)
train_labels = pd.read_csv("../input/amex-default-prediction/train_labels.csv")
train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
9,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-12-04,0.868580,0,0.001930,1.007504,0.001772,0.100470,0.0,0.006073,...,-1,-1,-1,0,0,0.0,NaN,0,0.004753,0
10,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-01-11,0.909811,0,0.002829,1.004798,0.008175,0.098882,0.0,0.003238,...,-1,-1,-1,0,0,0.0,NaN,0,0.003004,0
11,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-02-21,0.937349,0,0.019837,1.008307,0.000607,0.102985,0.0,0.007454,...,-1,-1,-1,0,0,0.0,NaN,0,0.006346,0
12,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-03-13,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.0,0.007174,...,-1,-1,-1,0,0,0.0,NaN,0,0.002970,0
22,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2017-12-29,0.861109,10,0.006711,0.819772,0.007966,0.139138,0.0,0.005235,...,-1,-1,-1,0,0,0.0,NaN,0,0.003133,0


In [31]:
#shapes
train.shape, test.shape, train_labels.shape

((1802318, 190), (3656852, 190), (458913, 2))

In [32]:
#Checking Missing values having more than 40%
columns = train.columns[(train.isna().sum()/len(train))*100>40]

In [33]:
#Drop the missing columns having more than 40%
train = train.drop(columns, axis=1)
test = test.drop(columns, axis=1)

In [34]:
#fill the missing values
train = train.bfill(axis='rows').ffill(axis='rows')
test = test.bfill(axis='rows').ffill(axis='rows')

In [35]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [36]:
train =train.groupby('customer_ID').tail(1)
test = test.groupby('customer_ID').tail(1)

In [37]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [38]:
#shape
train.shape, train_labels.shape, test.shape

((458913, 173), (458913, 2), (924621, 173))

In [39]:
#Type Conversion
obj_col = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
for col in obj_col:
    train[col]=train[col].astype('int').astype('str')
    test[col]=test[col].astype('int').astype('str')
    print(train[col].unique())
    print(test[col].unique())

['0' '1' '2' '-1']
['0' '1' '2' '-1']
['2' '1' '3' '7' '5' '6' '4' '-1']
['2' '3' '6' '1' '7' '5' '4' '-1']
['1' '0' '-1']
['0' '1' '-1']
['0' '-1' '1']
['0' '-1' '1']
['5' '0' '7' '2' '3' '6' '4' '-1']
['0' '4' '5' '3' '7' '6' '-1' '2']
['0' '1' '-1']
['1' '0' '-1']
['2' '1']
['1' '2']
['0' '3' '4' '5' '1' '2']
['0' '3' '4' '2' '1' '5']
['0' '2' '3' '-1']
['3' '0' '2' '-1']
['-1' '1']
['-1' '1']
['6' '3' '5' '4' '2' '1' '-1']
['6' '4' '5' '2' '1' '3' '-1']


**Data Preparation**

In [40]:
train = train.merge(train_labels, how='inner', on="customer_ID")

In [41]:
train.head()

,index,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,...,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145,target
0,12,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-03-13,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.0,...,-1,-1,-1,0,0,0.0,0,0.002970,0,0
1,25,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2018-03-25,0.880519,6,0.034684,1.004028,0.006911,0.165509,0.0,...,-1,-1,-1,0,0,0.0,0,0.003169,0,0
2,38,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2018-03-12,0.880875,0,0.004284,0.812649,0.006450,0.369885,0.0,...,-1,-1,-1,0,0,0.0,0,0.000834,0,0
3,51,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,2018-03-29,0.621776,0,0.012564,1.006183,0.007829,0.287766,0.0,...,-1,-1,-1,0,0,0.0,0,0.005560,0,0
4,64,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2018-03-30,0.871900,0,0.007679,0.815746,0.001247,0.463104,0.0,...,-1,-1,-1,0,0,0.0,0,0.006944,0,0


In [42]:
test_data = test.copy()
train = train.drop(['index','customer_ID', 'S_2'], axis=1)
test = test.drop(['index','customer_ID', 'S_2'], axis=1)

In [43]:
#one hot Encoding for categorical features
obj_col = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
train = pd.get_dummies(train, columns=obj_col, drop_first=True)
test = pd.get_dummies(test, columns=obj_col, drop_first=True)
train.shape, test.shape

((458913, 199), (924621, 198))

In [44]:
Features=train.loc[:, test.columns]
target = train['target']

In [45]:
from sklearn import preprocessing

df_train = Features.copy()

scaler = preprocessing.MinMaxScaler()

names = df_train.columns
dtrain = scaler.fit_transform(df_train)
normalized_train = pd.DataFrame(dtrain, columns=names)
normalized_train.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,...,D_64_0,D_64_2,D_64_3,D_66_1,D_68_1,D_68_2,D_68_3,D_68_4,D_68_5,D_68_6
0,0.948771,0.000000,0.731437,0.997671,0.002212,0.119434,0.0,0.004494,0.008502,0.030303,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.911856,0.035294,0.736606,0.994087,0.002504,0.125089,0.0,0.003174,0.006506,0.030303,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.912098,0.000000,0.730396,0.804603,0.002337,0.162997,0.0,0.004507,0.005983,0.030303,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.735715,0.000000,0.732087,0.996221,0.002837,0.147766,0.0,0.006224,0.004946,0.030303,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.905989,0.000000,0.731089,0.807669,0.000452,0.180288,0.0,0.003463,0.004792,0.030303,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [46]:
test

df_test = test.copy()

scaler = preprocessing.MinMaxScaler()

names = df_test.columns
dtest = scaler.fit_transform(df_test)
normalized_test = pd.DataFrame(dtest, columns=names)
normalized_test.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,...,D_64_0,D_64_2,D_64_3,D_66_1,D_68_1,D_68_2,D_68_3,D_68_4,D_68_5,D_68_6
0,0.690217,0.012618,0.428810,0.999354,0.0000,0.150470,0.0,0.002128,0.000764,0.027778,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.881428,0.012618,0.431325,0.999252,0.0000,0.142567,0.0,0.006777,0.013171,0.027778,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.780533,0.000000,0.424767,0.802052,0.0000,0.154027,0.0,0.009486,0.010872,0.027778,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.651065,0.034700,0.489149,0.203641,0.0000,0.157219,0.0,0.296705,0.021849,0.083333,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.469363,0.082019,0.669251,0.037644,0.1674,0.154484,0.0,0.494507,0.007422,0.111111,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [47]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier

**Model Building and Evaluation**

In [48]:
# XGB = XGBClassifier(n_estimators=10, max_depth=6, learning_rate=0.1)#.fit(Features, target)

knn = KNeighborsClassifier(n_neighbors=3,algorithm="ball_tree",n_jobs=-1)

# svm = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)

In [49]:
# np.mean(cross_val_score(XGB, Features, target, scoring='accuracy', cv=5))

np.mean(cross_val_score(knn, Features, target, scoring='accuracy', cv=5))

# np.mean(cross_val_score(svm, Features, target, scoring='accuracy', cv=5))

In [50]:
# XGB = XGBClassifier(n_estimators=300, max_depth=6, learning_rate=0.1).fit(Features, target)

knn = KNeighborsClassifier(n_neighbors=5).fit(normalized_train, target)

# svm = SGDClassifier(loss="hinge", penalty="l2").fit(Features, target)

In [51]:
# y_pred = XGB.predict(Features)
# y_pred = knn.predict(Features)
# y_pred = svm.predict(Features)

# accuracy_score(target, y_pred)

In [52]:
#AUC (ROC curve)
# fpr, tpr, thresholds = roc_curve(target, y_pred)
# roc_auc = auc(fpr, tpr)
# display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name=svm)
# display.plot()
# plt.show()

In [53]:
# print(classification_report(target, y_pred))

**Submission to Kaggle**

In [54]:
# test_data['prediction']=XGB.predict_proba(test)[:,1]
test_data['prediction']=knn.predict(normalized_test)
# test_data['prediction']=svm.predict(normalized_test)

test_data[['customer_ID','prediction']].to_csv("submission.csv", index=False)

KeyboardInterrupt: 